# CHAPTER 12 Advanced pandas（高级pandas用法）

# 12.1 Categorical Data（类别数据）

这一节会介绍pandas的Categorical类型。

# 1 Background and Motivation（背景和动力）

表格中的列克可能会有重复的部分。我们可以用unique和value_counts，从一个数组从提取不同的值，并计算频度：

In [4]:
import numpy as np
import pandas as pd

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

对于不同的类型数据值，一个更好的方法是用维度表（dimension table）来表示，然后用整数键（integer keys）来指代维度表：

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

用take方法来重新存储原始的，由字符串构成的Series：

In [8]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种用整数表示的方法叫做类别（categorical）或字典编码（dictionary-encoded）表示法。表示不同类别值的数组，被称作类别，字典，或层级。本书中我们将使用类别（categorical and categories）来称呼。表示类别的整数值被叫做，类别编码（category code），或编码（code）。

# 2 Categorical Type in pandas（pandas中的Categorical类型）

pandas中有一个Categorical类型，是用来保存那些基于整数的类别型数据。考虑下面的例子：

In [1]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [2]:
N = len(fruits)
N

8

In [5]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,8,0.296658
1,1,orange,9,1.160006
2,2,apple,12,1.090123
3,3,apple,9,1.017549
4,4,apple,3,0.405369
5,5,orange,6,3.453955
6,6,apple,9,2.733937
7,7,apple,8,1.255206


这里，df['fruit']是一个python的字符串对象。我们将其转换为类型对象：

In [13]:
fruits_cat = df['fruit'].astype('category') # 转为可编码的分类
fruits_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

fruits_cat的值并不是一个numpy数组，而是一个pandas.Categorical实例：

In [16]:
c = fruits_cat.values
type(c)

pandas.core.categorical.Categorical

这个Categorical对象有categories和codes属性：

In [17]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [18]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

可以把转换的结果变为DataFrame列：

In [6]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

也可以直接把其他的python序列变为pandas.Categorical类型：

In [10]:

print(df.fruit.values.categories)
print(df.fruit.values.codes)

Index(['apple', 'orange'], dtype='object')
[0 1 0 0 0 1 0 0]


In [20]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

如果已经得到了分类编码数据（categorical encoded data），我们可以使用from_codes构造器：

In [21]:
categories = ['foo', 'bar', 'baz']

In [22]:
codes = [0, 1, 2, 0, 0, 1]

In [23]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

除非明确指定，否则默认类别没有特定的顺序。所以，取决于输入的数据，categories数组可能有不同的顺序。当使用from_codes或其他一些构造器的时候，我们可以指定类别的顺序：

In [24]:
ordered_cat = pd.Categorical.from_codes(codes, categories, 
                                        ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

输出的结果中，`[foo < bar < baz]`表示foo在bar之间，以此类推。一个没有顺序的类型实例（unordered categorical instance）可以通过as_ordered来排序：

In [25]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

最后一点需要注意的，类型数据没必要一定是字符串，它可以是任何不可变的值类型
（any immutable value types）。

# 3 Computations with Categoricals（类型计算）

Categorical类型和其他类型差不多，不过对于某些函数，比如groupby函数，在Categorical数据上会有更好的效果。很多函数可以利用ordered标记。

假设有一些随机的数字，用pandas.quct进行分箱（binning）。得到的类型是pandas.Categorical；虽然之前用到过pandas.cut，但是没有具体介绍里面的细节：

In [26]:
np.random.seed(12345)

In [27]:
draws = np.random.randn(1000)

In [28]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

计算分箱后的分位数：

In [29]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.95, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.95, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

具体分位数并不如季度的名字直观，我们直接在qcut中设定labels：

In [30]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [31]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

bins caetegorical并没有包含边界星系，我们可以用groupby来提取：

In [32]:
bins = pd.Series(bins, name='quartile')

In [34]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


quartile列包含了原始的类别信息，包含bins中的顺序：

In [35]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### Better performance with categoricals （使用categoricals得到更好的效果）

使用categorical能让效果提高。如果一个DataFrame的列是categorical类型，使用的时候会减少很多内存的使用。假设我们有一个一千万的元素和一个类别：

In [36]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

把labels变为categorical：

In [37]:
categories = labels.astype('category')

可以看到labels会比categories使用更多的内存：

In [38]:
labels.memory_usage()

80000080

In [39]:
categories.memory_usage()

10000272

当然，转换成category也是要消耗计算的，不过这种消耗是一次性的：

In [40]:
%time _ = labels.astype('category')

CPU times: user 303 ms, sys: 70.1 ms, total: 373 ms
Wall time: 385 ms


在categories上使用groupby会非常快，因为用的是基于整数的编码，而不是由字符串组成的数组。

# 4 Categorical Methods（类别方法）

如果是包含categorical数据的Series数据，有和Series.str类似的一些比较特殊的方法。对于访问categories和code很方便：

In [41]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [42]:
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

属性cat可以访问categorical方法：

In [43]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [44]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

假设我们知道实际的类别超过了当前观测到的四个类别，那么我们可以使用set_categories方法来扩展：

In [45]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

数据本身似乎没有改变，不过在对其进行操作的时候会反应出来。例如，value_counts：

In [46]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [47]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

在大型数据集，categoricals经常用来作为省内存和提高效果的工具。在对一个很大的DataFrame或Series进行过滤后，很多类型可能不会出现在数据中。我们用remove_unused_categories方法来除去没有观测到的类别：

In [48]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [49]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

下面是一些categorical的方法：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/kbedp.png)

### Creating dummy variables for modeling（为建模创建哑变量）

在使用机器学习的一些工具时，经常要转变类型数据为哑变量（dummy variables ），也被称作是独热编码（one-hot encoding）。即在DataFrame中，给一列中不同的类别创建不同的列，用1表示出现，用0表示未出现。

例子：

In [50]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

在第七章也介绍过，pandas.get_dummies函数会把一维的类型数据变为包含哑变量的DataFrame：

In [51]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 12.2 Advanced GroupBy Use（高级GroupBy用法）

我们已经在第十章讨论了groupby的一些用法，这里还有一些技巧可能会用得到。

# 1 Group Transforms and “Unwrapped” GroupBys（组变换和无包装的GroupBy）

在第十章里，使用apply方法在组上进行转换操作的。还有一个内建的方法叫transform，和apply相同，但是在一些函数的用法上有一些限制：

- 可以产生一个标量，将数据广播（broadcast）到与组一样的形状（这里的broadcast可以理解为改变数据形状的方法，感兴趣的可以直接搜索 numpy broadcast）
- 可以产生一个和输入的组一样形状的对象
- 不能对输入进行改变

举个例子：

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


通过key来计算组的平均值：

In [3]:
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

假设我们想要产生一个和df['value']一样大小的Series，不过要用key分组后的平均值来替换。我们可以把函数lambda x: x.mean()给transform：

In [4]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

对于内建的聚合函数，我们可以传入一个字符串别名，就像使用groupby agg方法的时候一样：

In [5]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

就像apply，transform能用那些返回Series的函数，但是结果的大小和输入的必须一样。例如，我们通过一个lambda函数令每个小组都乘2：

In [6]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

一个更复杂的例子，我们可以按降序来计算每一个组：

In [7]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

考虑一个包含简单聚合的分组转换函数：

In [8]:
def normalize(x):
    return (x - x.mean()) / x.std()

使用transform或apply，都能得到一样的结果：

In [9]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [10]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

内建的聚合函数，比如mean, sum经常比一般的apply函数要快。而是用transform的话，会更快一些。这就需要我们使用无包装的组操作（upwrapped group operation）：

In [11]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [13]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

一个无包装的组操作可能会涉及多个组聚合操作，不过向量化操作会胜过这种操作。

# 2 Grouped Time Resampling（分组时间重采样）

对于时间序列数据，resample方法是一个基于时间的组操作。这里有一个样本表格： 

In [15]:
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times, 'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


我们用time索引，然后重采样：

In [16]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


假设一个DataFrame包含多个时间序列，用多一个key列来表示：

In [17]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N), 
                    'value': np.arange(N * 3.)})
df2[:7]

,key,time,value
0,a,2017-05-20 00:00:00,0.0
1,b,2017-05-20 00:00:00,1.0
2,c,2017-05-20 00:00:00,2.0
3,a,2017-05-20 00:01:00,3.0
4,b,2017-05-20 00:01:00,4.0
5,c,2017-05-20 00:01:00,5.0
6,a,2017-05-20 00:02:00,6.0


想要对key列的值做重采样，我们引入pandas.TimeGrouper对象：

In [18]:
time_key = pd.TimeGrouper('5min')

然后设置time为索引，对key和time_key做分组，然后聚合：

In [19]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [20]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


使用TimeGrouper的一个限制是时间必须是Series或DataFrame的索引才行。

# 12.3 Techniques for Method Chaining（方法链接的技巧）

对序列进行转换的时候，我们会发现会创建很多再也不会用到的临时变量（temporary variable）。比如下面的例子：

    df = load_data()
    df2 = df[df['col2'] < 0]
    df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
    result = df2.groupby('key').col1_demeaned.std()
    
这里我们不使用任何真实数据，这个例子说明了一些新方法。首先，DataFrame.assign方法是一个函数，它可以作为列赋值方法`df[k] = v`的替代品。它不会修改原有的对象，而是会返回一个带有修改标识的新DataFrame对象。所以下面两个方法是相等的：

    # Usual non-functional way 
    df2 = df.copy() 
    df2['k'] = v

    # Functional assign way 
    df2 = df.assign(k=v)

在原始对象上直接进行赋值比用assign会更快一些，但是assign可以使用更方便的方法链接（method chaining）:

    result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
             .groupby('key')
             .col1_demeaned.std())

需要记住的是，当使用方法链接的时候，你可能会需要引用临时对象。在之后的例子，我们不能引用load_data的结果，除非它被赋值给临时变量df. 。为了做到这一点，assign和其他一些pandas函数接受像函数一样函数参数（function-like arguments），也被称作为可调用（callables）。

为了演示callables，考虑上面例子里的一个片段：

    df = load_data() 
    df2 = df[df['col2'] < 0]

这句可以被写为：

    df = (load_data()[lambda x: x['col2'] < 0])

在这里，load_data的结果没有赋值给参数，所以传入`[]`中的函数被绑定到了绑定到了在某个链接状态下的对象上（so the function passed into [] is then bound to the object at that stage of the method chain）。

我们可以把整个序列携程一行链接表达式：

    result = (load_data()
              [lambda x: x.col2 < 0]
              .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
              .groupby('key')
              .col1_demeaned.std())

我们可以把代码写成这种风格，但也可以分解成为步来写，这样可读性会更高。

# 1 The pipe Method（pipe方法）

我们可以利用pandas内建的函数和一些用callables实现的方法链接，做很多事情。不过，有时候我们想要用自己的函数或一些第三方库里的函数。这就是pipe方法出现的原因。

假设有一系列函数调用：

    a = f(df, arg1=v1) 
    b = g(a, v2, arg3=v3) 
    c = h(b, arg4=v4)
    
当使用函数来接受或返回Series或DataFrame对象的时候，我们可以把上面的利用pipe重写为：

    result = (df.pipe(f, arg1=v1)
              .pipe(g, v2, arg3=v3) 
              .pipe(h, arg4=v4))

f(df)和df.pipe(f)是一样的，但是pipe能让链接调用更简单。

pipe一个有用的模式是生成一系列可重复的函数操作。例如，考虑计算两个组的平均值的差：

    g = df.groupby(['key1', 'key2']) 
    df['col1'] = df['col1'] - g.transform('mean')
    
假设我们想要能对不止一个组进行减值，只要改变分组键（group key）即可。除此之外，我们可能想要把这种转换用方法链接的形式实现。这里有一个例子：

    def group_demean(df, by, cols):
        result = df.copy() 
        g = df.groupby(by) 
        for c in cols:
            result[c] = df[c] - g[c].transform('mean') 
        return result
        
上面的也可以写为：

    result = (df[df.col1 < 0] 
              .pipe(group_demean, ['key1', 'key2'], ['col1']))